In [1]:
pip install pandas openpyxl fuzzywuzzy python-Levenshtein


Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import pandas as pd
import glob
from fuzzywuzzy import process


In [3]:
# Caminho da pasta onde estão os arquivos
pasta_infomarket = r'K:\Pricing\Indicadores de Promoção\Dados\Infomarket\Base detalhada'
pasta_smarket = r'K:\Pricing\Indicadores de Promoção\Dados\Infomarket\Encartes Smarket'

In [5]:
# 2. Função para pegar o arquivo mais recente com base em um padrão no nome
def arquivo_mais_recente(pasta, padrao):
    arquivos = glob.glob(os.path.join(pasta, padrao))
    if not arquivos:
        raise FileNotFoundError(f"Nenhum arquivo encontrado em {pasta} com o padrão: {padrao}")
    return max(arquivos, key=os.path.getmtime)

In [7]:
# Pegando arquivos de cada pasta
arquivo_infomarket = arquivo_mais_recente(pasta_infomarket, 'base_infomarket_2025_05_21*.csv')
arquivo_smarket = arquivo_mais_recente(pasta_smarket, 'Relatório_Importação_(sem_segmento)_-_Acordos_Comerciais-16053-21-05-2025-08_25*.csv')

In [9]:
print("Arquivo Infomarket:", arquivo_infomarket)
print("Arquivo Smarket:", arquivo_smarket)

Arquivo Infomarket: K:\Pricing\Indicadores de Promoção\Dados\Infomarket\Base detalhada\base_infomarket_2025_05_21.csv
Arquivo Smarket: K:\Pricing\Indicadores de Promoção\Dados\Infomarket\Encartes Smarket\Relatório_Importação_(sem_segmento)_-_Acordos_Comerciais-16053-21-05-2025-08_25.csv


In [11]:
# 4. Lê os arquivos em dataframes
df_infomarket = pd.read_csv(arquivo_infomarket, encoding='utf-8', sep=',', low_memory=False)

In [13]:
df_smarket = pd.read_csv(arquivo_smarket, encoding='utf-8', sep=';')

In [15]:
print(df_infomarket.columns)

Index(['leaflet_id', 'network', 'segment', 'state', 'city', 'leaflet_code',
       'leaflet_name', 'leaflet_type', 'leaflet_number_of_pages',
       'leaflet_last_images_upload', 'leaflet_created', 'leaflet_updated',
       'validity_start_date', 'validity_finish_date', 'period',
       'item_description', 'ean', 'price', 'minimum_quantity', 'dynamic',
       'requires_loyality', 'requires_credit_card', 'wholesale_only',
       'price_created', 'price_updated', 'item_measure', 'item_unit_measure',
       'category', 'supercategory', 'division', 'brand', 'manufacturer',
       'category_id', 'supercategory_id', 'images_url', 'channel_name'],
      dtype='object')


In [17]:
print(df_infomarket.head(5))

                 leaflet_id           network           segment  \
0  682b316bf485114637e70cb0  LÍDER ATACADISTA  VAREJO ALIMENTAR   
1  682b316bf485114637e70cb0  LÍDER ATACADISTA  VAREJO ALIMENTAR   
2  682b316bf485114637e70cb0  LÍDER ATACADISTA  VAREJO ALIMENTAR   
3  682b316bf485114637e70cb0  LÍDER ATACADISTA  VAREJO ALIMENTAR   
4  682b316bf485114637e70cb0  LÍDER ATACADISTA  VAREJO ALIMENTAR   

            state      city leaflet_code          leaflet_name leaflet_type  \
0  Santa Catarina    Laguna       e70cb0  PREÇO BAIXO TODO DIA      SEMANAL   
1  Santa Catarina  Imbituba       e70cb0  PREÇO BAIXO TODO DIA      SEMANAL   
2  Santa Catarina  Imbituba       e70cb0  PREÇO BAIXO TODO DIA      SEMANAL   
3  Santa Catarina  São José       e70cb0  PREÇO BAIXO TODO DIA      SEMANAL   
4  Santa Catarina   Tubarão       e70cb0  PREÇO BAIXO TODO DIA      SEMANAL   

   leaflet_number_of_pages leaflet_last_images_upload  ... item_unit_measure  \
0                        4   2025-05-19T13

In [19]:
print(df_smarket.columns)

Index(['SEQCAMPANHA', 'CAMPANHA', 'MIDIA', 'INICIO', 'FIM', 'SEQLOJA',
       'SEQOFERTA', 'TIPO_OFERTA', 'SEQPRODUTO', 'DESCRICAO', 'PRECO_VAREJO',
       'PRECO_ATACADO', 'QUANTIDADE_MINIMA', 'PRECO_FIDELIDADE',
       'SEQFORNECEDOR', 'NEGOCIACAO_ESPACO', 'TIPO_CASHBACK', 'CASHBACK_R$',
       'DESCONTO DE %', 'QTD. PAGUE', 'QTD. LEVE', 'COMPRA DE',
       'DESCONTO EM UNIDADES', 'PRECO_REGULAR', 'PÁGINA'],
      dtype='object')


In [21]:
#unindo o preço varejo e fidelidade e  transformando em apenas meu preço para trazer o menor
df_smarket['MEU_PRECO'] = df_smarket[['PRECO_VAREJO', 'PRECO_FIDELIDADE']].min(axis=1)

In [23]:
# Selecionando as colunas para dar match de cada base
concorrentes = df_infomarket[['item_description', 'price','network','leaflet_name','leaflet_type', 'validity_start_date','validity_finish_date', 'images_url']].dropna()
proprios = df_smarket[['DESCRICAO', 'MEU_PRECO', 'CAMPANHA','INICIO', 'FIM',]].dropna()

In [25]:
# Função para achar o melhor match fuzzy
def encontrar_melhor_match(produto, lista_produtos, threshold=90):
    match, score = process.extractOne(produto, lista_produtos)
    if score >= threshold:
        return match
    else:
        return None


In [27]:
#lista de produtos dos concorrentes para comparação
concorrentes_melhor_oferta = concorrentes.sort_values(by='price', ascending=True).drop_duplicates(subset='item_description', keep='first').copy()
#ordena os concorrentes por preço mais baixo primeiro

In [29]:
# 2. Criar a lista de produtos concorrentes para o fuzzy matching
lista_concorrentes = concorrentes_melhor_oferta['item_description'].tolist()

In [31]:
# 3. Criar o dicionário de concorrentes otimizado, usando a melhor oferta de cada produto
concorrentes_dict = concorrentes_melhor_oferta.set_index('item_description').to_dict('index')


In [33]:
#garantindo que está td certo 
print(concorrentes['item_description'].value_counts().head(10))


item_description
POTE SANREMO 785ML                                                 264
FRIGIDEIRA TRAMONTINA PLOTTER ESTAMPAS 1.2L 24CM                   264
POTE SANREMO 800ML                                                 258
JOGO DE ASSADEIRAS TRAMONTINA ANTIADERENTE ROSA 2 PEÇAS            223
KIT BULE SANREMO + SUPORTE PARA FILTRO DE CAFÉ N102 700ML CORES    220
CONJUNTO DE POTE SANREMO FÁCIL 360ML 4UND                          220
FAQUEIRO TRAMONTINA IPANEMA 24 PEÇAS                               220
FEIRINHA SANREMO UND                                               192
LASANHA SEARA SABORES 600G                                         184
FAQUEIRO TRAMONTINA LAGUNA INOX 16 PEÇAS                           180
Name: count, dtype: int64


In [35]:
# Lista para armazenar resultados
resultados = []

In [37]:
#lista de descrições únicas dos concorrentes
lista_concorrentes = list(concorrentes_dict.keys())

In [39]:
# Função objetiva para formatar em Reais
def formatar_para_reais(valor):
    # 4 espaços de identação para o corpo da função
    if pd.isna(valor):
        # 8 espaços de identação para o bloco 'if'
        return None 
    
    # 4 espaços de identação para as linhas de processamento
    s_valor = f"{valor:,.2f}"
    s_valor = s_valor.replace(",", "X").replace(".", ",").replace("X", ".")
    
    # 4 espaços de identação para a linha de retorno final
    return f"R$ {s_valor}"

In [41]:
#loop nos produtos próprios
for i, row in proprios.iterrows(): #iterrows = linha por linha
    produto_proprio = row['DESCRICAO'] #extração dos dados
    preco_proprio = row['MEU_PRECO']
    campanha = row['CAMPANHA']
    inicio_vigencia = row['INICIO']
    fim_vigencia = row['FIM']
    
    
    #Buscar melhor correspondencia com fuzzy matching    
melhor_match, score = process.extractOne(produto_proprio, lista_concorrentes) #encontrar_melhor_match busca o nome mais parecido com fuzzy matching
    # Inicializa variáveis para o caso de não haver match ou score baixo
preco_concorrente = None
indice_formatado = None
concorrente_mais_barato = None
rede_concorrente = None
nome_encarte = None
tipo_encarte = None
url_encarte = None
validity_start_date = None
validity_finish_date = None # <-- Adicione esta variável
diferenca_preco_bruta = None
diferenca_preco_reais_formatada = None

if score >= 90:
        info_concorrente = concorrentes_dict[melhor_match]
        preco_concorrente = info_concorrente ['price']
        rede_concorrente = info_concorrente ['network']
        validity_start_date_conc = info_concorrente.get('validity_start_date')
        validity_finish_date_conc = info_concorrente.get('validity_finish_date')
        nome_encarte = info_concorrente['leaflet_name']
        tipo_encarte = info_concorrente['leaflet_type']
        url_encarte = info_concorrente['images_url']
        
    # Evita divisão por zero
        if preco_concorrente is not None and preco_concorrente > 0:
            indice_valor = preco_proprio / preco_concorrente
            #formata para percentual com arredondamento e %
            indice_formatado = f'{round(indice_valor * 100)}%'
            diferenca_preco_bruta = preco_proprio - preco_concorrente
            diferenca_preco_reais_formatada = formatar_para_reais(diferenca_preco_bruta)
           
        else: #se preço concorrente for zero ou none
            indice_formatado = None
            diferenca_preco_bruta = None
            diferenca_preco_reais_formatada = None

        resultados.append({
            'Produto_Proprio': produto_proprio,
            'Preco_Proprio': preco_proprio,
            'Campanha_Propria': campanha,
            'INICIO_Proprio': inicio_vigencia,
            'FIM_Proprio': fim_vigencia,
            'Produto_Concorrente': melhor_match,
            'Preco_Concorrente': preco_concorrente,
            'Rede_Concorrente': info_concorrente['network'],
            'Nome_Encarte': info_concorrente['leaflet_name'],
            'Tipo_Encarte': info_concorrente['leaflet_type'],
            'URL_Encarte': info_concorrente['images_url'],
            'INICIO_Concorrente_Encarte': validity_start_date_conc, 
            'FIM_Concorrente_Encarte': validity_finish_date_conc,
            'Indice_Formatado': indice_formatado,
            'Diferenca_Preco_Reais': diferenca_preco_reais_formatada,
        })

In [43]:
# Criar dataframe dos resultados
df_resultado = pd.DataFrame(resultados)

In [45]:
print(df_resultado.head(5))

   Produto_Proprio  Preco_Proprio  \
0  LARANJA PERA KG           4.79   

                                    Campanha_Propria INICIO_Proprio  \
0  KOCH - CAPA ESPECIAL PALHOÇA, SÃO FRANCISCO DO...     2025-04-28   

  FIM_Proprio Produto_Concorrente  Preco_Concorrente      Rede_Concorrente  \
0  2025-05-02     LARANJA PERA KG               3.79  SUPERMERCADO MESCHKE   

  Nome_Encarte Tipo_Encarte  \
0  HORTI FRUTI   HORTIFRUTI   

                                         URL_Encarte  \
0  https://app.infomarketpesquisa.com/app/leaflet...   

  INICIO_Concorrente_Encarte FIM_Concorrente_Encarte Indice_Formatado  \
0                 20/05/2025              20/05/2025             126%   

  Diferenca_Preco_Reais  
0               R$ 1,00  


In [47]:
df_resultado.columns

Index(['Produto_Proprio', 'Preco_Proprio', 'Campanha_Propria',
       'INICIO_Proprio', 'FIM_Proprio', 'Produto_Concorrente',
       'Preco_Concorrente', 'Rede_Concorrente', 'Nome_Encarte', 'Tipo_Encarte',
       'URL_Encarte', 'INICIO_Concorrente_Encarte', 'FIM_Concorrente_Encarte',
       'Indice_Formatado', 'Diferenca_Preco_Reais'],
      dtype='object')

In [49]:
#reordenando as colunas
nova_ordem = ['Campanha_Propria','INICIO_Proprio', 'FIM_Proprio','Produto_Proprio', 'Preco_Proprio','Produto_Concorrente',
       'Preco_Concorrente', 'Indice_Formatado', 'Diferenca_Preco_Reais','Rede_Concorrente', 'Nome_Encarte', 'Tipo_Encarte',
       'URL_Encarte', 'INICIO_Concorrente_Encarte', 'FIM_Concorrente_Encarte']

In [51]:
#Reordenando
df_reordenado = df_resultado[nova_ordem]

In [57]:
print(df_reordenado)

                                    Campanha_Propria INICIO_Proprio  \
0  KOCH - CAPA ESPECIAL PALHOÇA, SÃO FRANCISCO DO...     2025-04-28   

  FIM_Proprio  Produto_Proprio  Preco_Proprio Produto_Concorrente  \
0  2025-05-02  LARANJA PERA KG           4.79     LARANJA PERA KG   

   Preco_Concorrente Indice_Formatado Diferenca_Preco_Reais  \
0               3.79             126%               R$ 1,00   

       Rede_Concorrente Nome_Encarte Tipo_Encarte  \
0  SUPERMERCADO MESCHKE  HORTI FRUTI   HORTIFRUTI   

                                         URL_Encarte  \
0  https://app.infomarketpesquisa.com/app/leaflet...   

  INICIO_Concorrente_Encarte FIM_Concorrente_Encarte  
0                 20/05/2025              20/05/2025  


In [59]:
# Salvar em Excel para análise
df_resultado.to_excel(r'K:\Pricing\Indicadores de Promoção\Dados\Infomarket\Automatizacao\IDENTICOS\Comparativo4.xlsx', index=False)
print("Análise concluída e salva em 'comparativo_encartes.xlsx'")

PermissionError: [Errno 13] Permission denied: 'K:\\Pricing\\Indicadores de Promoção\\Dados\\Infomarket\\Automatizacao\\IDENTICOS\\Comparativo4.xlsx'

In [49]:
#Preço Médio por rede de concorrente
df_resultado.groupby('Rede_Concorrente')['Preco_Concorrente'].mean()

Rede_Concorrente
SUPERMERCADO MESCHKE    3.79
Name: Preco_Concorrente, dtype: float64